In [2]:
using Distributions
using Optim

In [53]:
delta1, delta2 = -3 , -2
beta1, beta2 = [1 4], [2 3]
rho = -2
lambda = exp(rho)/(1+exp(rho))

N = 200
Y = zeros(N,2)
X = zeros(N,2)

io = open("monte_carlo.csv", "w")
println(io, "y1,y2,x1,x2")

for i=1:N
    x1,x2 = rand(Normal(0.0, 2.0),3)
    e1,e2 = rand(Logistic(),2)
    select = rand(Categorical([lambda, (1-lambda)]))
    
    #see logit response i.e. definitely 11
    eq = []
    for y_init=[[0 0],[1 1],[0 1],[1 0]]
        y1,y2 = sum(beta1.*[1 x1]) + delta1*y_init[2] + e1 >= 0, sum(beta2.*[1 x2]) +delta2*y_init[1] + e2 >= 0
        
        if false
            println( "$(1*y1) $(1*y2),$(y_init)" )
            println(sum(beta1.*[1 x1]) + delta1*y_init[2] + e1," ", sum(beta2.*[1 x2]) +delta2*y_init[1] + e2)
            println("-----")
        end
            
        if (y1==y_init[1]) & (y2==y_init[2])
            append!(eq,[y_init])
        end
    end
    #println( eq )
    #println( "========")
    y1, y2 = eq[1]
    if size(eq)[1] >= 2
        y1, y2 = eq[select]
    end
    Y[i,:] =[y1 y2]
    X[i,:] =[x1 x2]
    println(io, "$(y1),$(y2),$(x1),$(x2)")
end

close(io)

In [54]:
function ll_obj(params;xs=X,ys=Y)
    n = size(X)[1]
    k = Integer(size(X)[2]/2) + 1
    
    p = []
    
    for bound=[[0,0],[1,1],[0,1],[1,0]]
        util1 = params[1] .+ xs[:,1].*params[2]  .+ bound[2].*params[3]
        util2 = params[4] .+ xs[:,2].*params[5]  .+ bound[1].*params[6]
        prob = exp.(util1).^(bound[1])./(1 .+ exp.(util1))
        prob = prob .* exp.(util2).^(bound[2])./(1 .+ exp.(util2))
        append!(p,[prob])
    end
    
    
    p00 = (1 .- ys[:,1]).*(1 .- ys[:,2])
    p11 = ys[:,1].*ys[:,2]
    p10 = ys[:,1].* (1 .- ys[:,2])
    p01 = (1 .- ys[:,1]) .* ys[:,2]
    
    #print(ys[2,:], " ", p[1][2]," ",p[2][2], " ", p[3][2]," ",p[4][2] )
    #print(max.(p[1] .+ p[2].+p[4]) )
    
    lambda = exp(params[7])/(1+exp(params[7]))
    #println(1 .- p[1] .- p[2] .- p[4])
    ll = p00 .* log.(p[1]) + p11 .* log.(p[2])
    ll = ll .+ p01 .* log.(max.( lambda .* p[3] + (1-lambda).*(1 .- p[1] .- p[2] .- p[4] ),0 ) )
    ll = ll .+ p10 .* log.( max.( (1-lambda) .* p[4] + lambda.*(1 .- p[1] .- p[2] .- p[3] ),0 ) )
    return -sum(ll)    
end


ll_obj([1 4 -3 2 3 -2 -1];xs=X,ys=Y)

75.34621244689407

In [55]:
a = 10
b =(exp(a)/(1+exp(a)))^2 + 1/4 + 1/2*exp(a)/(1+exp(a))
println( (exp(a)/(1+exp(a)))^2 )
println( 1/2*exp(a)/(1+exp(a)) )
print(b)

0.9999092063235617
0.4999773010656488
1.7498865073892105

In [56]:
function maximum_likelihood(params)   
    return optimize(ll_obj, params, LBFGS())
end

result =  maximum_likelihood([1 4 -3 2 3 -2 -1.])
print(result.minimizer)
print(result.minimum)

[1.25651 3.57987 -2.70656 2.81621 3.61223 -2.53774 -27.855]73.00457344439384